___
# Ciência dos Dados - PROJETO 2 - 2019/1

___
## Daniel Castro

## Omar Dibo Calixto Afrange Neto
___

## Imports

In [7]:
import pandas as pd
import os
import numpy as np
from functools import reduce
from scipy import stats
import matplotlib.pyplot as plt

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [8]:
dados=pd.read_excel('spamham2019(1).xlsx')
x=dados.sample(frac=1)
x=x.sample(frac=1).reset_index(drop=True)

treinamento = x.loc[:5572*0.75,:]
teste = x.loc[5572*0.75:,:]

treinamento_spam = treinamento[treinamento.Class == "spam"]
treinamento_ham = treinamento[treinamento.Class == "ham"]

___
## 3. Classificador Naive-Bayes

**Limpando o DataFrame**

In [9]:
lista_treinamento = treinamento["Email"].tolist()
lista_treinamento = str(lista_treinamento)
lista_treinamento = lista_treinamento.replace(",","")
lista_treinamento = lista_treinamento.replace("'","")
lista_treinamento = lista_treinamento.replace("[","")
lista_treinamento = lista_treinamento.replace("]","")
lista_treinamento = lista_treinamento.replace(":","")
lista_treinamento = lista_treinamento.replace(";","")
lista_treinamento = lista_treinamento.replace("_","")
lista_treinamento = lista_treinamento.replace("$","")
lista_treinamento = lista_treinamento.replace("&","")
lista_treinamento = lista_treinamento.replace("(","")
lista_treinamento = lista_treinamento.replace(")","")
lista_treinamento = lista_treinamento.replace(".","")
lista_treinamento = lista_treinamento.replace("#","")                                              
lista_treinamento = lista_treinamento.split()

In [10]:
lista_treinamento_spam = treinamento_spam["Email"].tolist()
lista_treinamento_spam = str(lista_treinamento_spam)
lista_treinamento_spam = lista_treinamento_spam.replace(",","")
lista_treinamento_spam = lista_treinamento_spam.replace("'","")
lista_treinamento_spam = lista_treinamento_spam.replace("[","")
lista_treinamento_spam = lista_treinamento_spam.replace("]","")
lista_treinamento_spam = lista_treinamento_spam.replace(":","")
lista_treinamento_spam = lista_treinamento_spam.replace(";","")
lista_treinamento_spam = lista_treinamento_spam.replace("_","")
lista_treinamento_spam = lista_treinamento_spam.replace("$","")
lista_treinamento_spam = lista_treinamento_spam.replace("&","")
lista_treinamento_spam = lista_treinamento_spam.replace("(","")
lista_treinamento_spam = lista_treinamento_spam.replace(")","")
lista_treinamento_spam = lista_treinamento_spam.replace(".","")
lista_treinamento_spam = lista_treinamento_spam.replace("#","")                                              
lista_treinamento_spam = lista_treinamento_spam.split()

In [11]:
lista_treinamento_ham = treinamento_ham["Email"].tolist()
lista_treinamento_ham = str(lista_treinamento_ham)
lista_treinamento_ham = lista_treinamento_ham.replace(",","")
lista_treinamento_ham = lista_treinamento_ham.replace("'","")
lista_treinamento_ham = lista_treinamento_ham.replace("[","")
lista_treinamento_ham = lista_treinamento_ham.replace("]","")
lista_treinamento_ham = lista_treinamento_ham.replace(":","")
lista_treinamento_ham = lista_treinamento_ham.replace(";","")
lista_treinamento_ham = lista_treinamento_ham.replace("_","")
lista_treinamento_ham = lista_treinamento_ham.replace("$","")
lista_treinamento_ham = lista_treinamento_ham.replace("&","")
lista_treinamento_ham = lista_treinamento_ham.replace("(","")
lista_treinamento_ham = lista_treinamento_ham.replace(")","")
lista_treinamento_ham = lista_treinamento_ham.replace(".","")
lista_treinamento_ham = lista_treinamento_ham.replace("#","")                                              
lista_treinamento_ham = lista_treinamento_ham.split()

**Propabilidades de Ham e Spam**

In [12]:
numero_ham = treinamento['Email'][treinamento['Class'] == "ham"].count() #Contagem do total de hams 
numero_spam = treinamento['Email'][treinamento['Class'] == "spam"].count() #Contagem do total de spams
total = numero_ham + numero_spam #Contagem total de mensagens

porcentagem_ham= (numero_ham/total) #Probabilidade de hams
porcentagem_spam= (numero_spam/total) #Probabilidade de irrelevantes

print ("Porcentagem de hams:{0}%".format(porcentagem_ham*100))
print ("Porcentagem de spams:{0}%".format(porcentagem_spam*100))

Porcentagem de hams:86.4354066985646%
Porcentagem de spams:13.564593301435407%


**Contando o número de vezes que cada palavra aparece**

In [13]:
#Total
palavras_treinamento = {}
for palavra in lista_treinamento:
    if palavra not in palavras_treinamento:
        palavras_treinamento[palavra] = 1
    else:
        palavras_treinamento[palavra] += 1
        
        
#Spams        
palavras_treinamento_spam = {}
for palavra in lista_treinamento_spam:
    if palavra not in palavras_treinamento_spam:
        palavras_treinamento_spam[palavra] = 1
    else:
        palavras_treinamento_spam[palavra] += 1
        
        
#Hams
palavras_treinamento_ham = {}
for palavra in lista_treinamento_ham:
    if palavra not in palavras_treinamento_ham:
        palavras_treinamento_ham[palavra] = 1
    else:
        palavras_treinamento_ham[palavra] += 1

**Calculando a probabilidade da mensagem ser Ham ou Spam**

In [16]:
def classificador_mensagem(mensagem, dic1, dic2):
    
    probabilidade_spam = 0
    probabilidade_ham = 0
    mensagem_splitada = mensagem.split()
    soma_spam = sum(palavras_treinamento_spam.values())
    soma_ham = sum(palavras_treinamento_ham.values())
    resultado=str()
    for palavra in mensagem_splitada:
        quantidade_ham = 0
        quantidade_spam = 0
        if palavra in dic1:
            quantidade_ham += palavras_treinamento_ham[palavra]
        if palavra in dic2:
            quantidade_spam += palavras_treinamento_spam[palavra]
        probabilidade_ham = ((quantidade_ham)+1)/(soma_spam + len(palavras_treinamento))
        probabilidade_spam = ((quantidade_spam)+1)/(soma_ham + len(palavras_treinamento))
    if probabilidade_spam/probabilidade_ham >= 1:
        resultado = "spam"
    elif probabilidade_spam/probabilidade_ham < 1:
        resultado = "ham"
    return resultado


___
## 4. Qualidade do Classificador alterando a base de treinamento

In [15]:
#Verificar a porcentagem de acerto do classificador (precisão)
classificacao = []
for mensagem in teste["Email"]:
    classificacao.append(classificador_mensagem(mensagem, palavras_treinamento_ham, palavras_treinamento_spam))

teste["Classificador"] = classificacao

porcentagem_acerto = teste["Email"][teste["Class"] == teste["Classificador"]].count()
print(porcentagem_acerto/len(teste) * 100,"% de acerto")

90.66762383345298 % de acerto


C:\Users\danie\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Conclusão

O Classificador de Naive-Bayes desenvolvido por nós, para classificar e-mails, como 'úteis' ou 'spam', apresentou uma qualidade muito boa com 90,67% de acerto, porem isso nos leva, ao mesmo tempo, a desconfiar um pouco do mesmo, uma vez que a base de treinamento utilizada nao era tão ampla, o que poderia ser facilmente resolvido com um aumento da base de dados. Além disso poderíamos melhorar a classificação de relevância gerando uma menor taxa de erros. Se pudessemos implementar essas novas ideias talvez possamos no futuro trabalhar em um projeto mais complexo de reconhecimento de vírus em um computador ou smartphone por exemplo.